In [1]:
# -------------------------------------
# Master script for Maritime Risk Model
#
#         Ryan Lloyd
#         22/10/2018
# -------------------------------------

# ------- USAGE --------
#
# 1. Requires list of AIS data and incidents
# 2. Requires some static model parameters
#
# ----------------------

In [2]:
# Load environments
import os
import sys
import arcgis
import arcpy
import pandas
#import geopandas
import csv
import numpy
import matplotlib
import matplotlib.pyplot
import pylab
import glob
import zipfile
from arcpy.sa import *
from importlib import reload

In [3]:
# Input files
AIS123_root= r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/AIS_123/" # OS SPECIFIC "/"
AIS005_root= r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/AIS_005/" # OS SPECIFIC
incidents_root= r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/incidents/" # OS SPECIFIC
AIS123_file = "files_123.txt"
AIS005_file = "files_005.txt"
incidents_file = "incidents.txt"
continent_polygon = "c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/continents/World_Continents.lyrx"

FileGDB_path = "c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/"
FileGDB_name = "risk_model.gdb"

In [4]:
# Variables (FN size etc)

spatial_ref = arcpy.SpatialReference(4326) #GC_WGS_1984 - 4326, WC - 3857
AISjoin_field = "MMSI"
AISjoin_order = "ISODate"
grid_extent = [7, 10, -3, -7] # N,E,S,W
SIZE  = 2500   # AREA (km^2) of grid
SHAPE = "SQUARE" # Shape of grid cells

In [5]:
# ------ DO NOT EDIT BELOW HERE ----------

In [6]:
# Add folder of python modules:
sys.path.append(r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_functions/")

In [106]:
import load_data
import create_FileGDB
import traffic
import incidents
reload(incidents)
reload(load_data)
reload(create_FileGDB)
reload(traffic)

<module 'traffic' from 'C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_functions\\traffic.py'>

In [107]:
# Load local modules
from load_data import csv2shp, loadais, loadincidents, csv2dict
from create_FileGDB import creategdb
from traffic import ais2traffic
from create_grid import tes_grid
from incidents import incident_density

In [89]:
# Set up FileGDB
creategdb(FileGDB_path, FileGDB_name)
arcpy.env.workspace = FileGDB_path + FileGDB_name

FileGDB already exists


In [10]:
# Load in the AIS and incidents
incidentspts = csv2shp(incidents_root,incidents_file,FileGDB_path,FileGDB_name)
AIS123pts = csv2shp(AIS123_root, AIS123_file,FileGDB_path,FileGDB_name)

csv files converted to points:
incidents_022017.csv >> incidents_022017
csv files converted to points:
GoG123_20170201.csv >> GoG123_20170201
GoG123_20170202.csv >> GoG123_20170202
GoG123_20170203.csv >> GoG123_20170203


In [48]:
# Create Shipping tracks
AIS123pts_test = AIS123pts[:1]
traffic_lyr = ais2traffic(FileGDB_path,AIS123pts_test,AISjoin_field,AISjoin_order,continent_polygon)

Output traffic files:
c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170201_offshore_traffic_lyr
['c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170201_offshore_traffic_lyr']


In [50]:
# Create base grid
GRD = tes_grid(FileGDB_path, FileGDB_name, traffic_lyr, continent_polygon, grid_extent,spatial_ref, SIZE, SHAPE)

['c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/risk_model.gdb/GoG2500_grid_sea']

In [116]:
OUT, out_date = incident_density(FileGDB_path, FileGDB_name, incidentspts, GRD)
print(OUT)
print(out_date)

c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/risk_model.gdb/incident_density
['c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/risk_model.gdb/incident_density_022017']


In [14]:
import arcpy

# Set the workspace environment
arcpy.env.workspace ="c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/continents/"

# Get a list of the feature classes in the input folder
feature_classes = arcpy.ListFeatureClasses()

# Loop through the list
for fc in feature_classes:
    # Create the spatial reference object
    spatial_ref = arcpy.Describe(fc).spatialReference

    # If the spatial reference is unknown
    if spatial_ref.name == "Unknown":
        print("{0} has an unknown spatial reference".format(fc))

    # Otherwise, print out the feature class name and
    # spatial reference
    else:
        print("{0} : {1}".format(fc, spatial_ref.name))

In [ ]:
# Line density....? (parallel?)